# Deep Reinforcement Learning for Keras

<table>
  <tr>
    <td><img src="img/breakout.gif?raw=true" width="200"></td>
    <td><img src="img/cartpole.gif?raw=true" width="200"></td>
    <td><img src="img/pendulum.gif?raw=true" width="200"></td>
  </tr>
</table>


## What is it?

`keras-rl` implements some state-of-the art deep reinforcement learning algorithms in Python and seamlessly integrates with the deep learning library [Keras](http://keras.io).

Furthermore, `keras-rl` works with [OpenAI Gym](https://gym.openai.com/) out of the box. This means that evaluating and playing around with different algorithms is easy.

Of course you can extend `keras-rl` according to your own needs. You can use built-in Keras callbacks and metrics or define your own.
Even more so, it is easy to implement your own environments and even algorithms by simply extending some simple abstract classes. Documentation is available [online](http://keras-rl.readthedocs.org).



### Installations

In [ ]:
import keras
keras.__version__
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
print 'Running! \nPlease dont interrupt this cell. It might cause serious issues..'
!pip install gym keras-rl pyglet==1.2.4 
# !apt-get install -y cmake zlib1g-dev libjpeg-dev xvfb ffmpeg xorg-dev python-opengl libboost-all-dev libsdl2-dev swig 
!pip install 'gym[atari]' 
print 'Done!'

## Q-Learning with Neural Networks

Now, you may be thinking: tables are great, but they don’t really scale, do they? While it is easy to have a 16x4 table for a simple grid world, the number of possible states in any modern game or real-world environment is nearly infinitely larger. For most interesting problems, tables simply don’t work. 

We instead need some way to take a description of our state, and produce $Q$-values for actions without a table: that is where neural networks come in. By acting as a function approximator, we can take any number of possible states that can be represented as a vector and learn to map them to $Q$-values.

In the case of the FrozenLake example, we will be using a one-layer network which takes the state encoded in a one-hot vector (1x16), and produces a vector of 4 $Q$-values, one for each action. 

Such a simple network acts kind of like a glorified table, with the network weights serving as the old cells. The key difference is that we can easily expand the Tensorflow network with added layers, activation functions, and different input types, whereas all that is impossible with a regular table. 

<img src="images/pong.jpg" alt="" style="width: 300px;"/>

The method of updating is a little different as well. 

Instead of directly updating our table, with a network we will be using backpropagation and a loss function. Our loss function will be sum-of-squares loss, where the difference between the current predicted $Q$-values, and the “target” value is computed and the gradients passed through the network. 

In this case, our $Q_{target}$ for the chosen action is the equivalent to the $Q$-value computed in equation above ($
Q(s,a) + \alpha [r + \gamma max_{a'} Q(s',a') - Q(s,a) ]
$).

$$
Loss = \sum (Q_{target} - Q_{predicted})^2
$$

In [3]:
import gym
import numpy as np
from keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation, Flatten
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

Get the environment and extract the number of actions.

In [4]:
ENV_NAME = 'CartPole-v0'

env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

## Deep Q-networks

While our ordinary Q-network was able to barely perform as well as the Q-Table in a simple game environment, Deep $Q$-Networks are much more capable. In order to transform an ordinary Q-Network into a DQN we will be making the following improvements:
+ Going from a single-layer network to a multi-layer convolutional network.
+ Implementing Experience Replay, which will allow our network to train itself using stored memories from it’s experience.
+ Utilizing a second “target” network, which we will use to compute target $Q$-values during our updates.

<img src="images/deepq1.png" alt="" style="width: 800px;"/>


See https://jaromiru.com/2016/09/27/lets-make-a-dqn-theory/

### Convolutional Layers

Since our agent is going to be learning to play video games, it has to be able to make sense of the game’s screen output in a way that is at least similar to how humans or other intelligent animals are able to. Instead of considering each pixel independently, convolutional layers allow us to consider regions of an image, and maintain spatial relationships between the objects on the screen as we send information up to higher levels of the network.

### Experience Replay

The second major addition to make DQNs work is Experience Replay. 

The problem with online learning is that the *samples arrive in order* they are experienced and as such are highly correlated. Because of this, our network will most likely overfit and fail to generalize properly.

The key idea of **experience replay** is that we store these transitions in our memory and during each learning step, sample a random batch and perform a gradient descend on it. 

The Experience Replay buffer stores a fixed number of recent memories, and as new ones come in, old ones are removed. When the time comes to train, we simply draw a uniform batch of random memories from the buffer, and train our network with them. 

### Separate Target Network

This second network is used to generate the $Q$-target values that will be used to compute the loss for every action during training. 

The issue is that at every step of training, the $Q$-network’s values shift, and if we are using a constantly shifting set of values to adjust our network values, then the value estimations can easily spiral out of control. The network can become destabilized by falling into feedback loops between the target and estimated $Q$-values. In order to mitigate that risk, the target network’s weights are fixed, and only periodically or slowly updated to the primary $Q$-networks values. In this way training can proceed in a more stable manner.

Instead of updating the target network periodically and all at once, we will be updating it frequently, but slowly.

While the DQN we have described above could learn ATARI games with enough training, getting the network to perform well on those games takes at least a day of training on a powerful machine.

Next, we build a very simple model.

In [ ]:
inp = Input(shape=(1,) + env.observation_space.shape )
x = Flatten()(inp)
x = Dense(16)(x)
x = Activation('relu')(x)
x = Dense(16)(x)
x = Activation('relu')(x)
x = Dense(16)(x)
x = Activation('relu')(x)
x = Dense(nb_actions)(x)
x = Activation('linear')(x)
model = Model(inputs=inp, outputs=x)
model.summary()

Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
even the metrics!

## Introduction

---
<span style="float:right;">[[source]](https://github.com/keras-rl/keras-rl/blob/master/rl/agents/dqn.py#L89)</span>
### DQNAgent

```python
rl.agents.dqn.DQNAgent(model, policy=None, test_policy=None, enable_double_dqn=True, 
                       enable_dueling_network=False, dueling_type='avg')
```

In [ ]:
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()

In [ ]:
#single
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [6]:
#dual
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               enable_dueling_network=True, dueling_type='avg', target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

# Renderer 
for live animation in jupyter while running those command.
in case you are running on your local device you can comment the bottom lines and and run full local environment! (It looks nicer :) )

In [ ]:
class Render(Callback):
    def on_step_end(self, step, logs={}):
        plt.clf()
        plt.imshow(env.render(mode='rgb_array'))
        display.display(plt.gcf())
        display.clear_output(wait=True)

Okay, now it's time to learn something! We visualize the training here for show, but this
slows down training quite a lot. You can always safely abort the training prematurely using
Ctrl + C.

In [ ]:
# nb_steps represents the number of steps, you can try and change it
dqn.fit(env, callbacks=[Render()], nb_steps=1000, log_interval=10000)

If you are running locally, uncomment this out!

In [ ]:
# #nb_steps represents the number of steps, you can try and change it
# dqn.fit(env, nb_steps=100, visualize=True, verbose=2)

After training is done, we save the final weights.

In [6]:
dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

Finally, evaluate our algorithm for 5 episodes.

In [ ]:
dqn.test(env, nb_episodes=5, callbacks=[Render()], visualize=False)

If you are running locally, uncomment this out!

### SARSA & Expected SARSA

Let's say that:

$$X_t \in \{ v_t, \hat{v_t} \}$$

where in expected SARSA:

$$ v_t = r_t + \gamma \sum_a \pi_t (s_{t+1}, a) Q_t (s_{t+1}, a)$$

and in SARSA:

$$ \hat{v_t} = r_t + \gamma Q_t (s_{t+1}, a_{t+1})$$

Bias is represented by:

$$Bias(s,a) = Q^{\pi} (s,a) - E\{X_t\}$$

Variance is denoted by:

$$Var(s,a) = E\{(X_t)^2\} - (E\{X_t\})^2$$


In [ ]:
import gym
import numpy as np
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.layers import Input, Dense, Activation, Flatten
from rl.agents import SARSAAgent
from rl.policy import BoltzmannQPolicy

Get the environment and extract the number of actions.

In [3]:
ENV_NAME = 'CartPole-v0'

env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

Next, we build a very simple model.

In [ ]:
inp = Input(shape=(1,) + env.observation_space.shape)
x = Flatten()(inp)
x = Dense(16)(x)
x = Activation('relu')(x)
x = Dense(16)(x)
x = Activation('relu')(x)
x = Dense(16)(x)
x = Activation('relu')(x)
x = Dense(nb_actions)(x)
x = Activation('linear')(x)
model = Model(inputs=inp, outputs=x)
model.summary()

SARSA does not require a memory.

In [7]:
policy = BoltzmannQPolicy()
sarsa = SARSAAgent(model=model, nb_actions=nb_actions, nb_steps_warmup=10, policy=policy)
sarsa.compile(Adam(lr=1e-3), metrics=['mae'])

Okay, now it's time to learn something! We visualize the training here for show, but this
slows down training quite a lot. You can always safely abort the training prematurely using
Ctrl + C.

In [ ]:
# nb_steps represents the number of steps, you can try and change it
dqn.fit(env, callbacks=[Render()], nb_steps=1000, log_interval=10000)

If you are running locally, uncomment this out!

In [ ]:
# #nb_steps represents the number of steps, you can try and change it
# sarsa.fit(env, nb_steps=10000, visualize=True, verbose=2)

After training is done, we save the final weights.

In [9]:
sarsa.save_weights('sarsa_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

Finally, evaluate our algorithm for 5 episodes.

In [ ]:
dqn.test(env, nb_episodes=5, callbacks=[Render()], visualize=False)

If you are running locally, uncomment this out!

In [ ]:
# sarsa.test(env, nb_episodes=5, visualize=True)

### References
- [Playing Atari with Deep Reinforcement Learning](https://arxiv.org/abs/1312.5602), Mnih et al., 2013
- [Human-level control through deep reinforcement learning](http://www.nature.com/nature/journal/v518/n7540/abs/nature14236.html), Mnih et al., 2015
- [Deep Reinforcement Learning with Double Q-learning](http://www0.cs.ucl.ac.uk/staff/d.silver/web/Applications_files/doubledqn.pdf), van Hasselt et al., 2015
- [Dueling Network Architectures for Deep Reinforcement Learning](https://arxiv.org/abs/1511.06581), Wang et al., 2016